# Computational Experiment 1: Calculating Potentials


In this experiment, you will use some python code to numerically integrate the potential for some spherically-symmetric density distributions.

Before you begin, **make sure to read through the full notebook and understand what each function does**. Pay special attention to anything that has a **FIXME** note, which you will need to edit. Also make sure that you have ``numpy``, ``scipy``, ``galpy``, ``astropy``, and ``matplotlib`` installed (we don't technically need ``astropy`` for this one, but we will soon and it is an essential package to have if you are doing anything astro-related in python).

## 0. Import packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from galpy import potential as pot
from scipy.integrate import quad

## 1. Define density and analytic potential for Plummer distribution.

Note that ``galpy`` works best in "natural" units where $G=1$, so in this experiment we will use that convention. We will be more explicity about using physical units in future experiments. Also note that I've assumed $b=1$ and $M=1$.

In [ ]:
# Functions that define the density and potential for a variety of density profiles.
def rho_plummer(r):
    "Density of a Plummer model: BT equation 2.44b."
    return 3.0*(1.0 + r*r)**(-2.5) / (4.0*np.pi)

def pot_plummer(r):
    "Analytic potential of a Plummer model: BT equation 2.44a."
    # Note that the factor of G is missing here. We will be doing a lot of comparing
    # to galpy in this exercise, which uses dimensionless units where G=1, so we will
    # adopt those too for now.
    return -1.0/np.sqrt(r*r+1.0)

## 2. Routine to calculate potential numerically from a density distribution.

In [ ]:
def pot_numerical(r, dens):
    "Numerically integrate the potential from the function dens at radii r."
    
    # first define two convenience functions for the integration, which return the
    # density times r or times r squared.
    def rho_times_r(r):
        "rho(r) r. Used as the integrand for part of BT equation 2.28"
        return dens(r)*r
        
    def rho_times_r2(r):
        "rho(r) r^2. Used as the integrand for part of BT equation 2.28"
        return dens(r)*r*r
    
    # inner_integral is the first integral from BT equation 2.28
    inner_integral = np.array([quad(rho_times_r2, 0.0, x)[0] for x in r])
    # outer_integral is the second integral from BT equation 2.28
    outer_integral = np.array([quad(rho_times_r, x, np.inf)[0] for x in r])
    
    # BT equation 2.28: -4 pi G (inner_integral/r + outer_integral)
    # (but using G=1)
    return -4.0 * np.pi * (inner_integral/r + outer_integral)

## 3. Set up what you want to plot. These can be changed to plot different functions as you define them later on.

Note that there are **FIXME**s in all of these that you will want to come back to later.

In [ ]:
# FIXME: TO CHANGE TO A DIFFERENT DENSITY DISTRIBUTION, REPLACE
# THESE WITH THE NAMES OF YOUR DENSITY FUNCTION AND ANALYTIC
# POTENTIAL FUNCTION (AND AN APPROPRIATE LABEL).
# IF YOU DON'T HAVE AN ANALYTIC FORM OF THE POTENTIAL, YOU CAN SAY pot_func = False
rho_func = rho_plummer
pot_func = pot_plummer
label = 'Plummer'

In [ ]:
# FIXME: ONCE YOU'VE CONVINCED YOURSELF THAT YOUR NUMERICAL INTEGRATION VERSION OF THE
# POTENTIAL WORKS, YOU DON'T NEED TO PLOT BOTH IT AND GALPY'S VERSION, SO YOU CAN
# SET THIS TO False.
plot_my_pot_numerical = True

In [ ]:
# FIXME: TO USE ONE OF GALPY'S BUILT-IN POTENTIALS, PUT IT HERE.
# IF THERE IS NO BUILT-IN VERSION, YOU CAN SAY pot_galpy = False
pot_galpy = pot.PlummerPotential(amp=1.0, b=1.0)

In [ ]:
# galpy has a built-in option to create a potential from a density distribution
pot_galpy_from_density = pot.AnySphericalPotential(dens=rho_func)

## 4. Plot potential and density profile.

In [ ]:
# create an array that contains the values along the x axis
xax = 10**np.arange(-2, 3, 0.05)

In [ ]:
# Create a figure showing the potential:
# Numerical potential
if plot_my_pot_numerical:
    plt.plot(xax, pot_numerical(xax, rho_func), 'b.', label=label+' (numerical)')
# add a line for the analytic potential if it exists
if pot_func:
    plt.plot(xax, pot_func(xax), 'y--', label=label+' (analytic)')
# add a line for the galpy potential if it exists
if pot_galpy:
    plt.plot(xax, pot_galpy(xax,0), 'r-.', label=label+' (galpy built-in)')
# add a line for galpy's numerical solution given the density
plt.plot(xax, [pot_galpy_from_density(x,0) for x in xax], 'k-', ms=2, label=label+' (galpy numerical)')
# Make the x scale logarithmic
plt.xscale('log')

# add axis labels
plt.xlabel('$r$')
plt.ylabel('$\Phi$')
# add the legend
plt.legend(loc='lower right')
# save the figure to a file
plt.savefig(label+'_potential.png')

In [ ]:
# Create a figure showing the density:
plt.figure()
# log-log plot of the density
plt.loglog(xax, rho_func(xax), 'k-', label=label)
# add a line for galpy's built-in if it exists
if pot_galpy:
    plt.plot(xax, pot_galpy.dens(xax,0), 'r-.', label=label+' (galpy built-in)')
# add axis labels
plt.xlabel('$r$')
plt.ylabel('$\\rho$')
# add the legend
plt.legend()
# save the figure to a file
plt.savefig(label+'_density.png')

**Look at these plots!** What do you learn from them? Does the density go to the expected limits at $r \ll 1$ and $r \gg b$?

Note that these figures are saved as PNG files with names ``Plummer_potential.png`` and ``Plummer_density.png``.

## 5. Calculate the circular velocity and escape velocity

First it is convenient to define a helper function that integrates the enclosed mass given a density distribution. You can call this function from within your functions.

In [ ]:
def enclosed_mass(r, dens):
    "Enclosed mass"
    
    # first define a convenience function for the integration, which returns the
    # density times r squared
    def rho_times_r2(r):
        "rho(r) r^2. Used as the integrand for part of BT equation 2.28"
        return dens(r) * r * r
        
    # integral is from BT equation 2.27b
    integral = np.array([quad(rho_times_r2, 0.0, x)[0] for x in r])
    return 4.0 * np.pi * integral

**FIXME:** Finish the ``vesc`` function so that it calculates the escape velocity for a potential (BT equation 2.31) and the ``vcirc`` function so that it calculates the circular velocity (BT equation 2.29; you might find it convenient to call the ``enclosed_mass`` function from within ``vcirc``).

In [ ]:
def vesc(r, pot):
    "Escape veloctiy"
    # FIXME: PUT IN YOUR CODE HERE
    
def vcirc (r, rho):
    "Circular velocity"
    # FIXME: PUT IN YOUR CODE HERE.
    # YOU MIGHT FIND THE enclosed_mass FUNCTION HELPFUL, WHICH RETURNS THE ENCLOSED MASS
    # WITHIN A GIVEN RADIUS FOR A DENSITY FUNCTION.

In [ ]:
# Create a figure showing the circular velocities:
if plot_my_pot_numerical:
    plt.plot(xax, vcirc(xax, rho_func), 'k-', label='$v_{circ}$ '+label)
# add a line for the galpy numerical solution
plt.plot(xax, [pot_galpy_from_density.vcirc(x) for x in xax], 'b.', ms=2, label='$v_{circ}$ '+label+' (galpy numerical)')
# add a line for galpy's built in if it exists
if pot_galpy:
    plt.plot(xax, pot_galpy.vcirc(xax), 'y--', label='$v_{circ}$ '+label+' (galpy built-in)')
# Make x axis logarithmic
plt.xscale('log')
plt.xlabel('$r$')
plt.ylabel('$v$')
plt.legend()
plt.savefig(label+'_vc.png')

In [ ]:
# same for vesc
plt.figure()
if pot_func:
    plt.plot(xax, vesc(xax, pot_func), 'k-', label='$v_{escape}$ '+label)
else:
    if plot_my_pot_numerical:
        plt.plot(xax, vesc(xax, pot_numerical(xax, rho_func)), label='$v_{escape}$ '+label)
# add a line for the galpy numerical solution
plt.plot(xax, [pot_galpy_from_density.vesc(x) for x in xax], 'b.', ms=2, label='$v_{escape}$ '+label+' (galpy numerical)')
# add a line for galpy's built-in if it exists
if pot_galpy:
    plt.plot(xax, pot_galpy.vesc(xax), 'y--', label='$v_{escape}$ '+label+' (galpy built-in)')
# add axis labels
plt.xscale('log')
plt.xlabel('$r$')
plt.ylabel('$v$')
# add legend
plt.legend()
# save to file
plt.savefig(label+'_vesc.png')

**Look at these plots!** What do you learn from them?

Note that these figures are saved as PNG files with names ``Plummer_vc.png`` and ``Plummer_vesc.png``.

## 6. Try different mass distributions

**FIXME:** First, implement an NFW distribution in the ``rho_NFW`` and ``pot_NFW`` functions below.

In [ ]:
def rho_NFW(r):
    "Density of an NFW model: BT equation 2.64."
    # FIXME: PUT IN YOUR CODE HERE
    
def pot_NFW(r):
    "Analytic potential of an NFW model: BT equation 2.67."
    # FIXME: PUT IN YOUR CODE HERE

Now go back up to section 3 and update ``rho_func``, ``pot_func``, ``label``, and ``pot_galpy`` appropriately (you may want to look at [the galpy documentation on potentials](https://docs.galpy.org/en/v1.7.0/reference/potential.html#specific-potentials)) and re-run all of the code to generate plots for the NFW distribution. You might find it most convenient to create new notebook cells here and copy the code from section 3 so that you don't overwrite what you did above.

**Look at the plots.** How is the NFW density distribution different from the Plummer distribution? How are the potentials different? How are the circular and escape velocities different? How easily do you think you could observationally tell the difference between them?

## 7. Try something

Finally, try implementing another density distribution for which you don’t know the analytic form of the potential (note: if there is no analytic potential, then set ``pot_func=False``). I encourage you to be creative rather than making it physically realistic! Pay attention to how sensitive the shape of the potential and of the velocities is to the shape of the density distribution, and think about what that means for observationally determining density distributions.

**Everyone please bring your plots**, not just the discussion leader, especially for the last part where everyone’s should look different!